# Probabilistic Regression with NGBoost To Predict Uncertainty

The goal of regression models is to make accurate point predictions with input from independent variables. However, in the real world, the actual value most likely ends up being higher or lower than this point prediction. Estimating the uncertainty in the predictions with a full probability distribution over the target space allows data scientists and stakeholders to analyze a range of likely values for the outcome, and probabilistic forecasting can quantify these uncertainties.

Here, I use the NGBoost algorithm to predict a probability distribution of the outcome on the Ames housing dataset. NGBoost employs gradient boosting methods, which have been among the top performers in prediction accuracy for structured data. Three components are configured in NGBoost: (1) Base Learner (usually DecisionTree); (2) Probability Distribution (Normal, Log-Normal, Gamma etc); (3) Scoring rule (Maximum Likelihood Estimation (MLE), RMSE etc). For my model, I use DecisionTree as the base learner, Gamma distribution, and MLE as the scoring rule.

In [1]:
#pip install --upgrade git+https://github.com/stanfordmlgroup/ngboost.git

In [2]:
from ngboost import NGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
from ngboost.distns import Exponential, Normal, LogNormal, Gamma
from scipy.stats import lognorm, norm, gamma
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
import math
from scipy import stats
import shap
import numpy as np
import re
import os
import glob

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [3]:
ames_housing = pd.read_csv("datasets/AmesHousing.csv")
scoreid    = 533210020  # PID of select home

In [4]:
pd.set_option('display.max_columns', None)
ames_housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [5]:
# Random seed values
seeds = [324343, 454325, 465365, 787867, 645767,
         235436, 135436, 976876, 452454, 867356]

In [6]:
def x_y_split(df):
    X = df.drop(['SalePrice', 'Order', 'PID'], axis=1)
    y = df['SalePrice']
    return X, y

In [7]:
# Split data into X and y
# Hold out home-of-interest only

def dataset_split(data, start = 0, end = 360000000):
    # set up constraint
    scoring = data[data.PID==scoreid].reset_index(drop = True)
    data = data[(data.SalePrice>= start)&(data.SalePrice<=end)].reset_index(drop = True)
    X_id = list(data['PID'].unique())
    
    dataset = data[data.PID.isin(X_id)].reset_index(drop=True)
    dataset = dataset[dataset.PID!=scoreid].reset_index(drop=True)
    
    x, y = x_y_split(dataset)
    x_score, y_score = x_y_split(scoring)
    
    return x, y, dataset, scoring, x_score, y_score

In [8]:
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import clone

In [9]:
# Simple wrapper to set up the regressor

def initialize_reg(n_estimators     = 500, 
                   learning_rate    = 0.1,
                   minibatch_frac   = 1.0,
                   col_sample       = 1.0,
                   min_samples_leaf = 1,
                   max_depth        = 3,
                   dist             = Normal,
                   verbose = False,
                   random_state = None):
    reg = NGBRegressor( Base           = DecisionTreeRegressor(
                                            min_samples_leaf = min_samples_leaf,
                                            max_depth        = max_depth,
                                            random_state     = random_state ),
                       Dist = dist,
                        n_estimators   = n_estimators,
                        learning_rate  = learning_rate,
                        minibatch_frac = minibatch_frac,
                        col_sample     = col_sample,
                        random_state   = random_state,
                        verbose = verbose)
    return clone(reg)

In [10]:
# User inputs

data_dist = Gamma

param_dist = {
              'learning_rate': stats.uniform(0.009, 0.08),
              'minibatch_frac': stats.uniform(0.3, 0.7),
              'col_sample': stats.uniform(0.5, 0.45),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'min_samples_leaf': [1, 2, 3]
             }
n_param_samples = 6       # Number of random hyperparameter combinations to test
n_folds = 3                # Number of CV folds for each hyperparameter combination
max_n_estimators = 10000   # Make this a large number, early stopping will kick in
early_stopping_rounds = 10
random_state = 42

In [11]:
# SalePrice $ range

start_main = 0
end_main = 99999999999999  
start_alt = 80000
end_alt = 99999999999999

In [12]:
# Retrieve the best parameter set for a given metric and put it into a dict.

def get_search_result(param_set_results, param_list, metric = 'rmse'):
    
    best_param_set_index = \
        np.argmin([x['best_{}_score'.format(metric)] for x in param_set_results])
    best_param_set = param_list[best_param_set_index]
    best_param_set['n_estimators'] = \
        param_set_results[best_param_set_index]['best_{}_iteration'.format(metric)]
    best_score = param_set_results[best_param_set_index]['best_{}_score'.format(metric)]

    search_result = {}

    search_result['best_score'] = best_score
    search_result['best_param_set'] = best_param_set

    return(search_result)

In [13]:
def one_hot(df_imp, feat):
    # Fill missing values with "Missing"
    df_imp[feat] = df_imp[feat].fillna("Missing")
    
    # Initialize OneHotEncoder
    enc = OneHotEncoder()
    
    # Create a OneHotEncoder
    enc = OneHotEncoder(sparse_output=False)  # Set sparse=False to get a dense array

    # Fit and transform the encoder on the specified columns
    enc_data = enc.fit_transform(df_imp[feat])

    # Get the feature names from the encoder
    feature_names = enc.get_feature_names_out(input_features=feat)

    # Create a new DataFrame with proper column names
    enc_df = pd.DataFrame(enc_data, columns=feature_names)
        
    # Drop the original categorical columns from the original DataFrame
    df_imp = df_imp.drop(columns=feat)
    
    # Concatenate the original DataFrame with the encoded DataFrame
    final_df = pd.concat([df_imp, enc_df], axis=1)
    
    return final_df

In [14]:
def freq_enc(df, feat):
    
    # Perform frequency encoding for each categorical column
    for col in feat:
        # Fill missing values with "Missing"
        df[col].fillna("Missing", inplace=True)
        
        # Calculate frequencies of each category
        freq = df[col].value_counts().to_dict()
    
        # Replace the original values with frequencies
        df[col] = df[col].map(freq)
        
    return df

In [15]:
# Set up the fold data
folds = KFold(n_splits = n_folds, shuffle = True, random_state = random_state)
imp = SimpleImputer()

def folding(X, y):
    fold_data = list()
    for _, [train_index, test_index] in enumerate(folds.split(X)):
        X_train_fold, X_test_fold = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train_fold, y_test_fold = y.iloc[train_index].values.ravel(), \
                            y.iloc[test_index].values.ravel()
        
        # Impute missing values since ngboost doesn't handle missingness
        imp.fit(X_train_fold)
        X_train_fold = imp.transform(X_train_fold)
        X_test_fold  = imp.transform(X_test_fold)
        
        fold_data.append({'X_train'     : X_train_fold,
                          'y_train'     : y_train_fold,
                          'train_index' : train_index,
                          'X_test'      : X_test_fold,
                          'y_test'      : y_test_fold,
                          'test_index'  : test_index})
        
    return fold_data

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict

In [17]:
class PipelineJY(Pipeline):

    def pred_dist(self, X, **predict_params):
        """Transform the data, and apply `pred_dist` with the final estimator
        (NGBRegressor).
        Call `transform` of each transformer in the pipeline. The transformed
        data are finally passed to the final estimator that calls `pred_dist`
        method. Only valid if the final estimator implements `pred_dist`.
        Parameters
        ----------
        X : iterable
            Data to predict on. Must fulfill input requirements of first step
            of the pipeline.
        **predict_params : dict of string -> object
            Parameters to the ``pred_dist`` called at the end of all
            transformations in the pipeline. Note that while this may be
            used to return uncertainties from some models with return_std
            or return_cov, uncertainties that are generated by the
            transformations in the pipeline are not propagated to the
            final estimator.
        Returns
        -------
        y_pred : ndarray
            Result of calling `pred_dist` on the final estimator
        """
        Xt = X.copy()
        for _, name, transform in self._iter(with_final=False):
            Xt = transform.transform(Xt)      
        dists = self.steps[-1][1].pred_dist(Xt, **predict_params)
        ret_val = dists.params['alpha']
        ret_val = np.vstack([dists.params['alpha'],
                             dists.params['beta']]).T
        return ret_val

In [18]:
# Create df of predicted, min-max probability intervals, and observed values
def pred_dist_info(y, y_pred_params, data, pi_range = [0.05, 0.95]):
    
    alpha     = y_pred_params[:,0]
    beta = y_pred_params[:,1]
    
    y_pred_mean = alpha / beta
    y_pred_median = gamma.ppf(0.5, a = alpha, scale=1/beta)
    y_pred = y_pred_median
    
    errors = [y_pred -  gamma.ppf(pi_range[0], a = alpha, scale=1/beta),
              -y_pred + gamma.ppf(pi_range[1], a = alpha, scale=1/beta)]

    test = pd.DataFrame({"observe"     : y.values.flatten(),
                         "prediction"  : y_pred,
                         "error_minus" : errors[0],
                         "error_plus"  : errors[1],
                         "observed"    : y.values.flatten(),
                         "predicted"   : y_pred,
                         "lower_range" : y_pred - errors[0],
                         "upper_range" : y_pred + errors[1]}).reset_index(drop=True)
    
    for column in ['observed', 'predicted', 'lower_range', 'upper_range']:
        test[column] = test[column].apply(lambda x : '${0:,.0f}'.format(x))
    
    test = data['PID'].to_frame().merge(test, how='inner', left_index=True, right_index=True)
    test['held_out'] = np.where(test['PID']==scoreid, 'yes', 'no')
    
    return test, errors

In [19]:
def mask_outliers(df, replace):
    # Calculate Q1 and Q2 quantile
    q = df.agg('quantile', q=[.25, .75])

    # Calculate IQR = Q2 - Q1
    iqr = q.loc[.75] - q.loc[.25]

    # Calculate lower and upper limits to decide outliers
    lower = q.loc[.25] - 1.5 * iqr
    upper = q.loc[.75] + 1.5 * iqr

    # Replace the values that does not lies between [lower, upper]
    return df.where(df.ge(lower) & df.le(upper), replace)

In [20]:
def check_null(df):
    null_count = df.isnull().sum()
    null_pct = ((df.isnull().sum())/(df.isnull().count()))*100
    null_type = df.dtypes
    missing_data = pd.concat([null_count, null_pct, null_type], axis = 1,
                             keys = ['Null Values', 'Percent of Total', 'Data Type'])
    missing_data = missing_data.sort_values(by = 'Percent of Total', ascending = False).round(2)
    
    return missing_data

In [21]:
# Check for null values
check_null(ames_housing).head(20)

,Null Values,Percent of Total,Data Type
Pool QC,2917,99.56,object
Misc Feature,2824,96.38,object
Alley,2732,93.24,object
Fence,2358,80.48,object
Mas Vnr Type,1775,60.58,object
Fireplace Qu,1422,48.53,object
Lot Frontage,490,16.72,float64
Garage Cond,159,5.43,object
Garage Finish,159,5.43,object
Garage Yr Blt,159,5.43,float64


In [22]:
# Drop columns with >= 50% null values
ames_housing.drop(columns=['Pool QC', 'Misc Feature', 'Alley', 'Fence', 'Mas Vnr Type', 'Fireplace Qu'],
                  inplace=True)

# Replace outliers in numerical columns with NA
housing_num = mask_outliers(ames_housing.select_dtypes("number"), np.nan)

# Get list of numerical feature names
# Remove 'Order', 'PID', 'SalePrice'
num_feat = list(ames_housing.select_dtypes("number").columns.difference(['Order', 'PID', 'SalePrice']))

# Original number features
orig_num_feat = list(ames_housing.select_dtypes("number").columns)

# Get list of categorical feature names
cat_feat = list(ames_housing.columns.difference(orig_num_feat))
cat_feat_freq = ['Exterior 1st', 'Exterior 2nd', 'Neighborhood']
one_hot_feat = list(ames_housing.columns.difference(orig_num_feat).difference(cat_feat_freq))

# Join numerical and categorical columns
housing_cat = ames_housing[ames_housing.columns.difference(orig_num_feat)]
housing_new = housing_num.merge(housing_cat, how='inner', left_index=True, right_index=True)

In [23]:
# OneHotEncode categorical variables that each have <= 10 categories in df
# Treat NA's as a category
housing_new = one_hot(housing_new, one_hot_feat)

# FrequencyEncode categorical variables that each have > 10 categories in df
# Treat NA's as a category
housing_new = freq_enc(housing_new, cat_feat_freq)

# Split dataset
X, y, dataset, scoring, X_score, y_score = dataset_split(housing_new, start = start_main, end = end_main)

fold_data = folding(X, y)

best_nll_score = list()
nll_col_sample = list()
nll_learning_rate = list()
nll_max_depth = list()
nll_min_samples_leaf = list()
nll_minibatch_frac = list()
nll_n_estimators = list()
wmape = list()
house_sel_90_df = pd.DataFrame()
house_sel_80_df = pd.DataFrame()
house_sel_70_df = pd.DataFrame()
intervals_percent_df = pd.DataFrame()

In [24]:
check_null(housing_new).head(20)

,Null Values,Percent of Total,Data Type
Lot Frontage,677,23.11,float64
Enclosed Porch,459,15.67,float64
BsmtFin SF 2,352,12.01,float64
Screen Porch,256,8.74,float64
Overall Cond,252,8.60,float64
Mas Vnr Area,223,7.61,float64
MS SubClass,208,7.10,float64
Bsmt Half Bath,177,6.04,float64
Garage Yr Blt,162,5.53,float64
Open Porch SF,159,5.43,float64


In [25]:
# Drop Lot Frontage column
housing_new.drop(columns=['Lot Frontage'], inplace=True)

In [26]:
# Run the random search

# Two different metrics to minimize:
# RMSE and negative log likelihood
# With early stopping

def test_early_stop(metric_vs_stage, early_stopping_rounds):
    if len(metric_vs_stage) - np.argmin(metric_vs_stage) \
            > early_stopping_rounds:
        best_iteration = np.argmin(metric_vs_stage)
        best_test_score = metric_vs_stage[best_iteration]
        return((best_iteration, best_test_score))
    else:
        return((-1,-1))

def random_search(x, data_fold, best_nll_score, wmape_sel, col_sample, learning_rate, max_depth,
                  min_samples_leaf, minibatch_frac, n_estimators, sel_90_df, sel_80_df,
                  sel_70_df, intervals_percent_df, seeds, modelrun):
    param_list = list(ParameterSampler(param_dist,
                                       n_iter = n_param_samples,
                                       random_state = random_state))
    
    y_pred_train_folds = np.zeros(len(x))
    y_pred_test_folds  = np.zeros(len(x))
    y_pred_nll_train_folds = np.zeros(len(x))
    y_pred_nll_test_folds  = np.zeros(len(x))
    param_set_scores = np.zeros(len(param_list))
    param_set_results = list()
    for param_index, param_set in enumerate(param_list):
        print(param_set)
        
        # Initialize models for this hyperparameter set
        for fold in data_fold:
            fold['model'] = initialize_reg(**param_set, n_estimators = 1,
                                           dist=data_dist,
                                           random_state=seeds[modelrun])
            
        test_rmse_vs_stage = list()
        train_rmse_vs_stage = list()
        test_nll_vs_stage = list()
        train_nll_vs_stage = list()
        for i in range(max_n_estimators):
            
            for fold in data_fold:
                fold['model'].fit(fold['X_train'], fold['y_train'])
                
                # Get train- and test-fold predictions for this boosting stage
                y_pred_train_folds[fold['train_index']] = fold['model'].predict(fold['X_train'])
                y_pred_test_folds[fold['test_index']]   = fold['model'].predict(fold['X_test'])
                y_pred_nll_train_folds[fold['train_index']] = \
                    -fold['model'].pred_dist(fold['X_train']).logpdf(fold['y_train'])
                y_pred_nll_test_folds[fold['test_index']]   = \
                    -fold['model'].pred_dist(fold['X_test']).logpdf(fold['y_test'])
            
            # Compute metric at this number of trees
            stage_test_rmse  = mean_squared_error(y_pred_test_folds, y, squared=False)
            stage_train_rmse = mean_squared_error(y_pred_train_folds, y, squared=False)
            stage_test_nll  = y_pred_nll_test_folds.mean()
            stage_train_nll = y_pred_nll_train_folds.mean()
        
            test_rmse_vs_stage.append(stage_test_rmse)
            train_rmse_vs_stage.append(stage_train_rmse)
            test_nll_vs_stage.append(stage_test_nll)
            train_nll_vs_stage.append(stage_train_nll)
        
            if i % 10 == 0:
                print(('Iteration {}: RMSE - train folds {:,.2f}, test folds {:,.2f}' +
                       '\n\t\tNLL - train folds {:,.3f}, test folds {:,.3f}').\
                      format(i, stage_train_rmse, stage_test_rmse, 
                             stage_train_nll, stage_test_nll))
        
            # Stop if early stopping criterion is met
            best_rmse_iteration, best_test_rmse = test_early_stop(test_rmse_vs_stage,
                                                                  early_stopping_rounds)
            best_nll_iteration, best_test_nll   = test_early_stop(test_nll_vs_stage,
                                                                  early_stopping_rounds)

            if (best_rmse_iteration > 0) & (best_nll_iteration > 0):
                print('Best iterations: rmse - {:,.2f}, {:,.2f}\n\t\t nll - {:,.3f}, {:,.3f}'.format(
                    best_rmse_iteration, best_test_rmse, best_nll_iteration, best_test_nll))
                break
            
        param_set_results.append({'best_rmse_iteration': best_rmse_iteration,
                                  'best_rmse_score'    : best_test_rmse,
                                  'best_nll_iteration' : best_nll_iteration,
                                  'best_nll_score'     : best_test_nll})
    
    search_result = get_search_result(param_set_results, param_list, metric = 'nll')
    best_nll_score.append(search_result.get('best_score'))
    
    nll_best_param = list(search_result['best_param_set'].values())
    col_sample.append(nll_best_param[0])
    learning_rate.append(nll_best_param[1])
    max_depth.append(nll_best_param[2])
    min_samples_leaf.append(nll_best_param[3])
    minibatch_frac.append(nll_best_param[4])
    n_estimators.append(nll_best_param[5])
    
    # Set up steps and pipeline
    steps = [('imputer', SimpleImputer()),
             ('reg', initialize_reg(**search_result['best_param_set'],
                                    dist=data_dist,
                                    random_state=42))]
    pipeline = PipelineJY(steps)
    
    # Get 'alpha', 'beta', and predicted opening value
    y_pred_params_cv = cross_val_predict(pipeline, x, y, cv=folds, method="pred_dist")
    y_pred = cross_val_predict(pipeline, x, y, cv=folds, method="predict")
    
    # Create MAPE df and store MAPE
    mape_df = pd.DataFrame({'PID'            : dataset['PID'],
                            'SalePrice'      : y,
                            'predicted'      : y_pred,
                            'SellingPrice'   : y,
                            'predict'        : y_pred})
    
    for column in ['SellingPrice', 'predict']:
        mape_df[column] = mape_df[column].apply(lambda x : '${0:,.0f}'.format(x))
        
    wmape_run = round((np.abs(mape_df['SalePrice'].values - mape_df['predicted'].values).sum()
                      / np.abs(mape_df['SalePrice'].values).sum()), 7)
    wmape_sel.append(wmape_run)
    
    # Collect prediction and errors for home for each run and put in df
    # Middle 90%
    pred_dist_df, y_errors = pred_dist_info(y, y_pred_params_cv, dataset, pi_range = [0.05, 0.95])

    obs_percent_90 = np.mean(((y_pred + y_errors[1]) > pred_dist_df['observe']) &
                             ((y_pred - y_errors[0]) < pred_dist_df['observe'])) * 100

    sel_90_run = pred_dist_df[pred_dist_df['held_out']=='yes'
                                   ].drop('held_out', axis=1).reset_index(drop=True)
    sel_90_run['seed'] = modelrun + 1
    sel_90_run['type'] = 'middle_90'
    sel_90_df = pd.concat([sel_90_df, sel_90_run])

    # Middle 80%
    pred_dist_df, y_errors = pred_dist_info(y, y_pred_params_cv, dataset, pi_range = [0.1, 0.9])

    obs_percent_80 = np.mean(((y_pred + y_errors[1]) > pred_dist_df['observe']) &
                             ((y_pred - y_errors[0]) < pred_dist_df['observe'])) * 100

    sel_80_run = pred_dist_df[pred_dist_df['held_out']=='yes'
                            ].drop('held_out', axis=1).reset_index(drop=True)
    sel_80_run['seed'] = modelrun + 1
    sel_80_run['type'] = 'middle_80'
    sel_80_df = pd.concat([sel_80_df, sel_80_run])
    
    # Middle 70%
    pred_dist_df, y_errors = pred_dist_info(y, y_pred_params_cv, dataset, pi_range = [0.15, 0.85])

    obs_percent_70 = np.mean(((y_pred + y_errors[1]) > pred_dist_df['observe']) &
                             ((y_pred - y_errors[0]) < pred_dist_df['observe'])) * 100

    sel_70_run = pred_dist_df[pred_dist_df['held_out']=='yes'
                            ].drop('held_out', axis=1).reset_index(drop=True)
    sel_70_run['seed'] = modelrun + 1
    sel_70_run['type'] = 'middle_70'
    sel_70_df = pd.concat([sel_70_df, sel_70_run])
    
    # Create df of mean percent of intervals with observed value
    intervals_percent_run = pd.DataFrame.from_dict({'Middle_90%' : obs_percent_90,
                                                    'Middle_80%' : obs_percent_80,
                                                    'Middle_70%' : obs_percent_70
                                                   }, orient='index').T
    intervals_percent_df = pd.concat([intervals_percent_df, intervals_percent_run])
    
    return best_nll_score, wmape_sel, col_sample, learning_rate, max_depth, min_samples_leaf, minibatch_frac, n_estimators, \
            sel_90_df, sel_80_df, sel_70_df, intervals_percent_df, mape_df

In [27]:
for modelrun in range(10):
    best_nll_score, wmape, nll_col_sample, nll_learning_rate, nll_max_depth, nll_min_samples_leaf, nll_minibatch_frac, \
    nll_n_estimators, house_sel_90_df, house_sel_80_df, house_sel_70_df, intervals_percent_df, homes_df = \
    random_search(
        X, fold_data, best_nll_score, wmape, nll_col_sample, nll_learning_rate, nll_max_depth, nll_min_samples_leaf, \
        nll_minibatch_frac, nll_n_estimators, house_sel_90_df, house_sel_80_df, house_sel_70_df, intervals_percent_df, \
        seeds, modelrun)

{'col_sample': 0.6685430534813132, 'learning_rate': 0.0850571445127933, 'max_depth': 5, 'min_samples_leaf': 1, 'minibatch_frac': 0.7177951105625409}
Iteration 0: RMSE - train folds 56,893.86, test folds 56,870.34
		NLL - train folds 12.305, test folds 12.312
Iteration 10: RMSE - train folds 56,443.77, test folds 56,641.77
		NLL - train folds 12.291, test folds 12.304
Iteration 20: RMSE - train folds 57,100.01, test folds 57,123.76
		NLL - train folds 12.307, test folds 12.314
Best iterations: rmse - 10.00, 56,641.77
		 nll - 6.000, 12.302
{'col_sample': 0.700624738784116, 'learning_rate': 0.016997993265440228, 'max_depth': 5, 'min_samples_leaf': 1, 'minibatch_frac': 0.7207805082202461}
Iteration 0: RMSE - train folds 58,496.51, test folds 58,561.22
		NLL - train folds 12.348, test folds 12.351
Iteration 10: RMSE - train folds 58,437.08, test folds 58,469.93
		NLL - train folds 12.344, test folds 12.348
Iteration 20: RMSE - train folds 58,600.41, test folds 58,630.12
		NLL - train folds

Iteration 0: RMSE - train folds 58,384.50, test folds 58,471.03
		NLL - train folds 12.345, test folds 12.348
Iteration 10: RMSE - train folds 58,107.73, test folds 58,306.52
		NLL - train folds 12.337, test folds 12.344
Best iterations: rmse - 8.00, 58,229.56
		 nll - 8.000, 12.340
{'col_sample': 0.6943752583889521, 'learning_rate': 0.03229833121584335, 'max_depth': 5, 'min_samples_leaf': 3, 'minibatch_frac': 0.3976457024564293}
Iteration 0: RMSE - train folds 58,201.74, test folds 58,261.16
		NLL - train folds 12.339, test folds 12.342
Iteration 10: RMSE - train folds 58,252.29, test folds 58,437.05
		NLL - train folds 12.342, test folds 12.346
Iteration 20: RMSE - train folds 58,164.31, test folds 58,064.97
		NLL - train folds 12.340, test folds 12.338
Best iterations: rmse - 12.00, 58,027.61
		 nll - 12.000, 12.336
{'col_sample': 0.6314650918408482, 'learning_rate': 0.038308947463495335, 'max_depth': 8, 'min_samples_leaf': 2, 'minibatch_frac': 0.8496231729751094}
Iteration 0: RMSE 

Best iterations: rmse - 18.00, 58,680.25
		 nll - 18.000, 12.353
{'col_sample': 0.5818212352431953, 'learning_rate': 0.023672360788274706, 'max_depth': 6, 'min_samples_leaf': 2, 'minibatch_frac': 0.6673295021425665}
Iteration 0: RMSE - train folds 58,337.78, test folds 58,462.41
		NLL - train folds 12.344, test folds 12.348
Iteration 10: RMSE - train folds 58,377.95, test folds 58,515.42
		NLL - train folds 12.344, test folds 12.348
Iteration 20: RMSE - train folds 58,402.98, test folds 58,520.99
		NLL - train folds 12.343, test folds 12.349
Iteration 30: RMSE - train folds 58,395.69, test folds 58,450.47
		NLL - train folds 12.343, test folds 12.347
Best iterations: rmse - 21.00, 58,153.18
		 nll - 21.000, 12.339
{'col_sample': 0.6943752583889521, 'learning_rate': 0.03229833121584335, 'max_depth': 5, 'min_samples_leaf': 3, 'minibatch_frac': 0.3976457024564293}
Iteration 0: RMSE - train folds 58,372.61, test folds 58,415.36
		NLL - train folds 12.343, test folds 12.347
Iteration 10: RM

Iteration 30: RMSE - train folds 57,373.44, test folds 57,403.17
		NLL - train folds 12.315, test folds 12.322
Iteration 40: RMSE - train folds 57,133.76, test folds 57,734.55
		NLL - train folds 12.308, test folds 12.321
Best iterations: rmse - 31.00, 56,559.63
		 nll - 29.000, 12.294
{'col_sample': 0.700624738784116, 'learning_rate': 0.016997993265440228, 'max_depth': 5, 'min_samples_leaf': 1, 'minibatch_frac': 0.7207805082202461}
Iteration 0: RMSE - train folds 58,331.47, test folds 58,482.67
		NLL - train folds 12.342, test folds 12.347
Iteration 10: RMSE - train folds 58,548.13, test folds 58,485.92
		NLL - train folds 12.349, test folds 12.349
Iteration 20: RMSE - train folds 58,542.19, test folds 58,712.07
		NLL - train folds 12.348, test folds 12.353
Iteration 30: RMSE - train folds 58,712.15, test folds 58,717.18
		NLL - train folds 12.352, test folds 12.355
Best iterations: rmse - 29.00, 58,395.67
		 nll - 29.000, 12.344
{'col_sample': 0.8186326600082205, 'learning_rate': 0.0

In [28]:
intervals_percent_df = intervals_percent_df.reset_index(drop=True)
intervals_percent_df

,Middle_90%,Middle_80%,Middle_70%
0,96.203438,88.323782,79.154728
1,95.988539,92.084527,87.285100
2,89.756447,79.763610,70.702006
3,92.048711,81.769341,72.206304
4,96.704871,90.186246,81.411175
5,97.063037,91.547278,83.631805
6,97.242120,93.588825,87.679083
7,96.704871,90.186246,81.411175
8,93.624642,89.219198,83.703438
9,95.093123,86.461318,76.432665


In [29]:
intervals_percent_df.mean(axis=0)

Middle_90%    95.042980
Middle_80%    88.313037
Middle_70%    80.361748
dtype: float64

In [30]:
# Create df of best nll score and WMAPE
nll_wmape = pd.DataFrame.from_dict({'best_nll_score' : best_nll_score, 'WMAPE' : wmape}, orient='index').T
nll_wmape

,best_nll_score,WMAPE
0,12.302207,0.230756
1,12.304306,0.183591
2,12.306726,0.273433
3,12.302533,0.264649
4,12.300207,0.217767
5,12.303234,0.204617
6,12.295482,0.157612
7,12.302200,0.217767
8,12.293898,0.169728
9,12.299561,0.242239


In [31]:
nll_wmape_avg = nll_wmape.mean()
nll_wmape_avg

best_nll_score    12.301036
WMAPE              0.216216
dtype: float64

In [32]:
# House Prediction
# Apply the best parameter set for nll and assign it to reg
alpha_score = list()
beta_score = list()
y_pred_score = pd.DataFrame()

for modelrun in range(10):
    # Set up steps and pipeline
    steps = [('imputer', SimpleImputer()),
             ('reg', initialize_reg(n_estimators     = nll_n_estimators[modelrun],
                                    learning_rate    = nll_learning_rate[modelrun],
                                    minibatch_frac   = nll_minibatch_frac[modelrun],
                                    col_sample       = nll_col_sample[modelrun],
                                    min_samples_leaf = nll_min_samples_leaf[modelrun],
                                    max_depth        = nll_max_depth[modelrun],
                                    dist=data_dist,
                                    random_state=seeds[modelrun]))]
    pipeline = PipelineJY(steps)
    pipeline.fit(X, y)
    alpha_score.append(pipeline.pred_dist(X_score)[0][0])
    beta_score.append(pipeline.pred_dist(X_score)[0][1])
    
y_pred_score = pd.concat([pd.Series(alpha_score), pd.Series(beta_score)], axis=1)

In [33]:
# Create df of predicted, min-max probability intervals, and observed values
def pred_dist_info_score(y_pred_params, pi_range = [0.05, 0.95]):
    
    alpha     = y_pred_params.iloc[:,0].values
    beta = y_pred_params.iloc[:,1].values
    
    y_pred_mean = alpha / beta
    y_pred_median = gamma.ppf(0.5, a = alpha, scale=1/beta)
    y_pred = y_pred_median
    
    errors = [y_pred -  gamma.ppf(pi_range[0], a = alpha, scale=1/beta),
              -y_pred + gamma.ppf(pi_range[1], a = alpha, scale=1/beta)]

    test = pd.DataFrame({"prediction"  : y_pred,
                         "error_minus" : errors[0],
                         "error_plus"  : errors[1],
                         "alpha"       : alpha,
                         "beta"        : beta,
                         "predicted"   : y_pred,
                         "lower_range" : y_pred - errors[0],
                         "upper_range" : y_pred + errors[1]}).reset_index(drop=True)
    
    for column in ['predicted', 'lower_range', 'upper_range']:
        test[column] = test[column].apply(lambda x : '${0:,.0f}'.format(x))
    
    return test, errors

In [34]:
# Join nll_wmape df to score df
score_90_df, y_errors = pred_dist_info_score(y_pred_score, pi_range = [0.05, 0.95])
score_90_df = score_90_df.merge(nll_wmape, how='inner', left_index=True, right_index=True)
score_90_df

,prediction,error_minus,error_plus,alpha,beta,predicted,lower_range,upper_range,best_nll_score,WMAPE
0,162412.548961,71613.039568,102086.574049,9.887648,0.000059,"$162,413","$90,800","$264,499",12.302207,0.230756
1,169501.701835,53520.155525,67917.365599,21.511442,0.000125,"$169,502","$115,982","$237,419",12.304306,0.183591
2,162319.677517,76555.164842,112610.365306,8.394321,0.000050,"$162,320","$85,765","$274,930",12.306726,0.273433
3,162568.088621,75371.417386,109952.063726,8.753667,0.000052,"$162,568","$87,197","$272,520",12.302533,0.264649
4,161926.293682,70479.715446,99903.812775,10.200783,0.000061,"$161,926","$91,447","$261,830",12.300207,0.217767
5,164843.053723,69876.383887,97944.442796,10.867938,0.000064,"$164,843","$94,967","$262,787",12.303234,0.204617
6,166259.129555,60543.428525,80205.158396,15.527638,0.000091,"$166,259","$105,716","$246,464",12.295482,0.157612
7,164094.918195,71474.802534,101345.747097,10.183240,0.000060,"$164,095","$92,620","$265,441",12.302200,0.217767
8,161462.297795,41278.717846,49804.407623,34.435557,0.000211,"$161,462","$120,184","$211,267",12.293898,0.169728
9,162597.156224,73842.930026,106679.696992,9.205740,0.000055,"$162,597","$88,754","$269,277",12.299561,0.242239


In [35]:
def get_avg_df(df, score_range):
    df_avg = pd.DataFrame(df[['prediction', 'error_minus', 'error_plus',
                              'alpha', 'beta', 'best_nll_score', 'WMAPE']].mean()).T
    df_avg['predicted'] = df_avg['prediction']
    df_avg['lower_range'] = df_avg['prediction'] - df_avg['error_minus']
    df_avg['upper_range'] = df_avg['prediction'] + df_avg['error_plus']
    for column in ['predicted', 'lower_range', 'upper_range']:
        df_avg[column] = df_avg[column].apply(lambda x : '${0:,.0f}'.format(x))
    df_avg['actual'] = int(ames_housing[ames_housing['PID']==scoreid]['SalePrice'].iloc[0])
    df_avg['type'] = 'middle_{}'.format(score_range)
    df_avg['PID'] = scoreid
    return df_avg

In [36]:
score_range = 90
score_90_avg = get_avg_df(score_90_df, score_range)
score_90_avg

,prediction,error_minus,error_plus,alpha,beta,best_nll_score,WMAPE,predicted,lower_range,upper_range,actual,type,PID
0,163798.486611,66455.575559,92844.963436,13.896797,0.000083,12.301036,0.216216,"$163,798","$97,343","$256,643",173733,middle_90,533210020


In [37]:
# Join nll_wmape df to score df
score_80_df, y_errors = pred_dist_info_score(y_pred_score, pi_range = [0.1, 0.9])
score_80_df = score_80_df.merge(nll_wmape, how='inner', left_index=True, right_index=True)
score_80_df

,prediction,error_minus,error_plus,alpha,beta,predicted,lower_range,upper_range,best_nll_score,WMAPE
0,162412.548961,58196.077781,76717.527107,9.887648,0.000059,"$162,413","$104,216","$239,130",12.302207,0.230756
1,169501.701835,42869.264328,51613.821552,21.511442,0.000125,"$169,502","$126,632","$221,116",12.304306,0.183591
2,162319.677517,62460.935004,84379.813353,8.394321,0.000050,"$162,320","$99,859","$246,699",12.306726,0.273433
3,162568.088621,61430.006873,82451.168643,8.753667,0.000052,"$162,568","$101,138","$245,019",12.302533,0.264649
4,161926.293682,57234.107695,75117.027323,10.200783,0.000061,"$161,926","$104,692","$237,043",12.300207,0.217767
5,164843.053723,56664.005338,73721.492365,10.867938,0.000064,"$164,843","$108,179","$238,565",12.303234,0.204617
6,166259.129555,48751.262815,60695.987912,15.527638,0.000091,"$166,259","$117,508","$226,955",12.295482,0.157612
7,164094.918195,58044.456263,76198.992060,10.183240,0.000060,"$164,095","$106,050","$240,294",12.302200,0.217767
8,161462.297795,32863.313216,38040.543351,34.435557,0.000211,"$161,462","$128,599","$199,503",12.293898,0.169728
9,162597.156224,60109.821680,80069.490716,9.205740,0.000055,"$162,597","$102,487","$242,667",12.299561,0.242239


In [38]:
score_range = 80
score_80_avg = get_avg_df(score_80_df, score_range)
score_80_avg

,prediction,error_minus,error_plus,alpha,beta,best_nll_score,WMAPE,predicted,lower_range,upper_range,actual,type,PID
0,163798.486611,53862.325099,69900.586438,13.896797,0.000083,12.301036,0.216216,"$163,798","$109,936","$233,699",173733,middle_80,533210020


In [39]:
# Join nll_wmape df to score df
score_70_df, y_errors = pred_dist_info_score(y_pred_score, pi_range = [0.15, 0.85])
score_70_df = score_70_df.merge(nll_wmape, how='inner', left_index=True, right_index=True)
score_70_df

,prediction,error_minus,error_plus,alpha,beta,predicted,lower_range,upper_range,best_nll_score,WMAPE
0,162412.548961,48402.806020,60524.737763,9.887648,0.000059,"$162,413","$114,010","$222,937",12.302207,0.230756
1,169501.701835,35316.697929,41037.774933,21.511442,0.000125,"$169,502","$134,185","$210,539",12.304306,0.183591
2,162319.677517,52086.616559,66433.844034,8.394321,0.000050,"$162,320","$110,233","$228,754",12.306726,0.273433
3,162568.088621,51191.201974,64950.363454,8.753667,0.000052,"$162,568","$111,377","$227,518",12.302533,0.264649
4,161926.293682,47580.273142,59284.055537,10.200783,0.000061,"$161,926","$114,346","$221,210",12.300207,0.217767
5,164843.053723,47062.462862,58225.576955,10.867938,0.000064,"$164,843","$117,781","$223,069",12.303234,0.204617
6,166259.129555,40302.784050,48118.467935,15.527638,0.000091,"$166,259","$125,956","$214,378",12.295482,0.157612
7,164094.918195,48255.183316,60136.743015,10.183240,0.000060,"$164,095","$115,840","$224,232",12.302200,0.217767
8,161462.297795,26964.460168,30351.251575,34.435557,0.000211,"$161,462","$134,498","$191,814",12.293898,0.169728
9,162597.156224,50050.261922,63114.139914,9.205740,0.000055,"$162,597","$112,547","$225,711",12.299561,0.242239


In [40]:
score_range = 70
score_70_avg = get_avg_df(score_70_df, score_range)
score_70_avg

,prediction,error_minus,error_plus,alpha,beta,best_nll_score,WMAPE,predicted,lower_range,upper_range,actual,type,PID
0,163798.486611,44721.274794,55217.695512,13.896797,0.000083,12.301036,0.216216,"$163,798","$119,077","$219,016",173733,middle_70,533210020


In [41]:
score = pd.concat([score_90_avg, score_80_avg, score_70_avg]).reset_index(drop=True)
#score.to_csv('processed_data/10X_ngboost_gamma_score_inst.csv', index = False)

In [42]:
score

,prediction,error_minus,error_plus,alpha,beta,best_nll_score,WMAPE,predicted,lower_range,upper_range,actual,type,PID
0,163798.486611,66455.575559,92844.963436,13.896797,0.000083,12.301036,0.216216,"$163,798","$97,343","$256,643",173733,middle_90,533210020
1,163798.486611,53862.325099,69900.586438,13.896797,0.000083,12.301036,0.216216,"$163,798","$109,936","$233,699",173733,middle_80,533210020
2,163798.486611,44721.274794,55217.695512,13.896797,0.000083,12.301036,0.216216,"$163,798","$119,077","$219,016",173733,middle_70,533210020
